# Librerías

In [ ]:
pip install wget

In [1]:
import wget
import os
import zipfile
import pandas as pd
import csv
import time
import glob
import pathlib
import json
from sqlalchemy import create_engine
from os import remove
from os import rmdir
from shutil import rmtree
import shutil
from pathlib import Path
from scripts_aux import DownloadFipsCodes as dfc
from scripts_aux import Download_census_reports as dcr
from scripts_aux import Rename_columns_titles as rct

In [2]:
data = {'Name': ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming','American Samoa','Guam','Northern Mariana Islands','Puerto Rico','Virgin Islands'], 
        'Postal Code': ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY','AS','GU','MP','PR','VI'],
        'FIPS': [1,2,4,5,6,8,9,10,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,53,54,55,56,60,66,69,72,78]} 

dfFIPS = pd.DataFrame(data)

namelist = list()
FIPSlist = list()
for i in dfFIPS['Name']:
    namelist.append(i.lower())
for i in dfFIPS['FIPS']:
    FIPSlist.append(i)

In [35]:
table = 'B18101'
path = os.getcwd()
dirname = 'report'

In [37]:
#FIPSCode previo para optener el archivo metadata.json
indenttable = list()
nametable = list()
FIPSCode = str(FIPSlist[40])
# dfInicial, indenttable, nametable, listtitletable = dcr.download_census_previus_file(table, FIPSCode, path)
#dfInicial = dcr.download_census_total(dirname, table, FIPSlist, dfInicial)


In [38]:
url = 'https://api.dokku.censusreporter.org/1.0/data/download/acs2022_5yr?table_ids='+table+'&geo_ids=04000US'+FIPSCode+',01000US,860|04000US'+FIPSCode+'&format=csv'
zipname = '\\report'
wget.download(url, path+zipname+FIPSCode+'.zip')

zip_file = zipfile.ZipFile(path+zipname+FIPSCode+'.zip')
zip_file.extractall(path)
zip_file.close()
os.remove(path+zipname+FIPSCode+'.zip')

listtitletable = list()
with open(path+'\\acs2022_5yr_'+table+'_86000US57376\\metadata.json') as file:

        data = json.load(file)
        for i in data['tables']:
            nameprin = i

        for tables in data['tables'][nameprin]['columns']:
            listtitletable.append(tables)


indenttable = list()
nametable = list()
for i in listtitletable: 

        indenttable.append(data['tables'][nameprin]['columns'][i]['indent'])
        nametable.append(data['tables'][nameprin]['columns'][i]['name'])

    
raiz = path+'\\acs2022_5yr_'+table+'_86000US57376\\*.csv'
listcsv = glob.glob(raiz)

_df_prev = pd.read_csv(listcsv[0])
listcolumnsdataprev = _df_prev.columns
df = pd.DataFrame(columns = listcolumnsdataprev)

shutil.rmtree(path+'\\acs2022_5yr_'+table+'_86000US57376')
#return df, indenttable, nametable, listtitletable


In [ ]:
def download_census_total(dirname, table, _fips_list, df):
    os.mkdir(dirname)
    for i in range(len(_fips_list)):
        if(i > -1 and i < 7):

            _fips_code = str(_fips_list[i])
            _fips_code = '0'+_fips_code

        elif(i >= 7):        

            _fips_code = str(_fips_list[i])

        
        if i!=50 and i!=51 and i!=52 and i!=54 :
            
            path = os.getcwd()
            url = 'https://api.dokku.censusreporter.org/1.0/data/download/acs2022_5yr?table_ids='+table+'&geo_ids=04000US'+_fips_code+',01000US,860|04000US'+_fips_code+'&format=csv'
            print(url)
            wget.download(url, path+zipname+_fips_code+'.zip')

            zip_file = zipfile.ZipFile(path+zipname+_fips_code+'.zip')
            zip_file.extractall(path+zipname)
            zip_file.close()
            os.remove(path+zipname+_fips_code+'.zip')

    raiz = path+'\\'+dirname+'\\**'
    listruta = list()
    for ruta in glob.iglob(raiz):
        listruta.append(ruta)

    for i in range(len(listruta)):

        ruta = listruta[0]
        listruta.remove(ruta)
        nuevaruta = ruta.replace('/',"\\")
        listruta.append(nuevaruta)

    raiz = path+'\\'+dirname+'\**\*.csv'
    listcsv = glob.glob(raiz)

    for i in range(len(listruta)):
        
        _df_prev = pd.read_csv(listcsv[i])
        df = pd.concat([df,_df_prev], axis=0)

    shutil.rmtree(path+'\\'+dirname)

    return df

dfInicial = download_census_total(dirname, table, FIPSlist, df)

In [ ]:
dfInicial = rct.rename_titles(indenttable,dfInicial,nametable,listtitletable,'health_insurance_','health_')
dfFinal = dfInicial.dropna()
spike_cols = [col for col in dfFinal.columns if '_Error' not in col]


In [ ]:
dfFinal

In [43]:
dfFinal[dfFinal.columns.drop(list(dfFinal.filter(regex='_Error')))].to_excel('Table B18101 Sex by Age by Disability Status.xlsx')